<a href="https://colab.research.google.com/github/navrgithub/NLP_Authorship_Attribution/blob/main/Task1_Roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.6 MB/s eta 0:00:00


In [3]:
import pandas as pd
datapath = 'final_task1_data'
df = pd.read_csv(datapath)
df.head()

In [4]:
labels = [labels[label] for label in df['label']]
texts = [tokenizer(str(text), 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]

In [ ]:
labels = {'negative': 0, 'positive': 1}

In [ ]:
labels = [labels[label] for label in df['label']]

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
texts = [tokenizer(str(text), padding='max_length', max_length=512, truncation=True, return_tensors="pt") for text in df['text']]


In [ ]:
np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), 
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))
print(df_train.head())
print(df_val.head())


In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Define hyperparameters
batch_size = 32
num_epochs = 5
learning_rate = 2e-5


In [ ]:
# Create a TensorDataset and DataLoader for the train set
train_input_ids = torch.cat([text['input_ids'] for text in texts[:len(df_train)]], dim=0)
train_attention_mask = torch.cat([text['attention_mask'] for text in texts[:len(df_train)]], dim=0)
train_labels = torch.tensor(labels[:len(df_train)])


In [ ]:
train_dataset = TensorDataset(train_input_ids, train_attention_mask, train_labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


In [ ]:
# Create a TensorDataset and DataLoader for the validation set
val_input_ids = torch.cat([text['input_ids'] for text in texts[len(df_train):len(df_train)+len(df_val)]], dim=0)
val_attention_mask = torch.cat([text['attention_mask'] for text in texts[len(df_train):len(df_train)+len(df_val)]], dim=0)
val_labels = torch.tensor(labels[len(df_train):len(df_train)+len(df_val)])

In [ ]:
val_dataset = TensorDataset(val_input_ids, val_attention_mask, val_labels)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Initialize RoBERTa model
model = RobertaForSequenceClassification.from_pretrained('roberta-base')
model.to(device)


In [ ]:
# Set optimizer and loss function
optimizer = AdamW(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()


In [ ]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in data_loader:
        input_ids_batch, attention_mask_batch, labels_batch = tuple(t.to(device) for t in batch)
        optimizer.zero_grad()

        outputs = model(input_ids_batch, attention_mask=attention_mask_batch, labels=labels_batch)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

    avg_loss = total_loss / len(data_loader)
    print(f"Epoch {epoch+1}/{num_epochs} - Avg Loss: {avg_loss}")

# Save the trained model
model.save_pretrained("roberta_model")
tokenizer.save_pretrained("roberta_model")

In [ ]:
# Load and tokenize data
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
texts = ["Example sentence 1", "Example sentence 2", ...]  # List of input texts
labels = [0, 1, ...]  # List of corresponding labels (0 or 1)
encoded_inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
input_ids = encoded_inputs['input_ids']
attention_mask = encoded_inputs['attention_mask']
labels = torch.tensor(labels)


In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the trained model and tokenizer
model = RobertaForSequenceClassification.from_pretrained("roberta_model")
tokenizer = RobertaTokenizer.from_pretrained("roberta_model")

# Load the evaluation dataset and tokenize it
eval_texts = df_val['text'].tolist()
eval_labels = [labels[label] for label in df_val['label']]
encoded_inputs = tokenizer(eval_texts, padding=True, truncation=True, return_tensors="pt")
input_ids = encoded_inputs["input_ids"]
attention_mask = encoded_inputs["attention_mask"]
labels = torch.tensor(eval_labels)

# Create dataset and data loader for evaluation
eval_dataset = TensorDataset(input_ids, attention_mask, labels)
eval_loader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False)

# Evaluate the model
model.eval()
eval_loss, eval_acc, eval_steps = 0, 0, 0

with torch.no_grad():
    for batch in eval_loader:
        input_ids_batch, attention_mask_batch, labels_batch = tuple(t.to(device) for t in batch)

        outputs = model(input_ids_batch, attention_mask=attention_mask_batch, labels=labels_batch)
        loss, logits = outputs.loss, outputs.logits

        eval_loss += loss.item()
        eval_acc += (logits.argmax(1) == labels_batch).sum().item()
        eval_steps += 1

eval_loss /= eval_steps
eval_acc /= len(df_val)

print(f"Validation Loss: {eval_loss:.4f}")
print(f"Validation Accuracy: {eval_acc:.4f}")


In [ ]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids_batch, attention_mask_batch, labels_batch = tuple(t.to(device) for t in batch)
        optimizer.zero_grad()

        outputs = model(input_ids_batch, attention_mask=attention_mask_batch, labels=labels_batch)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {avg_loss}")